# SK Youtube Plugin

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [1]:
#r "nuget: dotenv.net"
#r "nuget: Microsoft.SemanticKernel, 1.49.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.49.0"
#r "nuget: System.Linq.Async, 6.0.1"
#r "nuget: YoutubeExplode, 6.5.0"

#r "nuget: dotenv.net"

using System;

using System.IO;
using System.Net.Http;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using System.ComponentModel;
using System.Threading;
using YoutubeExplode;
using YoutubeExplode.Common;

using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003
#pragma warning disable SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025
#pragma warning disable SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,
#pragma warning disable SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102


Installed Packages dotenv.net, 3.2.1 Microsoft.SemanticKernel, 1.49.0 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.49.0 System.Linq.Async, 6.0.1 YoutubeExplode, 6.5.0

### Read the API Key and endpoints from environment variables or the .env file

In [2]:
// Load the .env file
DotEnv.Load();

// Get the OpenAI deployment name, endpoint, and key from the environment variables
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var pg_conn_str = Environment.GetEnvironmentVariable("PG_CONN_STR");
var adaDeploymentName = Environment.GetEnvironmentVariable("GPT_EMBEDDING_MODEL");

### Define the lights plugin

In [3]:
record YouTubeVideo(string Title, string Url);
class YouTubePlugin
{
    [KernelFunction("get_youtube_video_captions")]
    [Description("Gets the English captions of a YouTube video")]
    [return: Description("The English captions of the YouTube video")]
    public async Task<string> GetVideoCaptionsAsync(
    [Description("The URL of the YouTube video")]
    string videoUrl, 
    CancellationToken cancellationToken)
    {
        // Create a YoutubeClient instance
        YoutubeClient youtube = new();
        var trackManifest = await youtube.Videos.ClosedCaptions.GetManifestAsync(videoUrl, cancellationToken);
        var trackInfo = trackManifest.GetByLanguage("en");
        var track = await youtube.Videos.ClosedCaptions.GetAsync(trackInfo, cancellationToken);

        StringBuilder sb = new();
        foreach (var caption in track.Captions)
        {
            sb.AppendLine(caption.Text);
        }
        
        return sb.ToString();
    }

    [KernelFunction("get_youtube_videos")]
    [Description("Gets the list of youtube videos from a channel given the channel's handle.")]
    [return: Description("The collection of YouTube videos that have been uploaded to the YouTube channel.")]
    public async Task<IReadOnlyList<YouTubeVideo>> GetVideosAsync(
    [Description("The channel's handle (e.g. @channelname)")]
    string channelHandle, 
    CancellationToken cancellationToken)
    {
        // Create a YoutubeClient instance
        YoutubeClient youtube = new();   
        channelHandle = channelHandle.TrimStart('@');
        var channel = await youtube.Channels.GetByHandleAsync(new YoutubeExplode.Channels.ChannelHandle(channelHandle), cancellationToken);
        var videos = await youtube.Channels
            .GetUploadsAsync(channel.Id, cancellationToken)
            .Where(x=>x.Duration> TimeSpan.FromMinutes(60))
            .Take(10)
            .ToArrayAsync(cancellationToken);
        return videos
        .Select(x=>new YouTubeVideo(x.Title,x.Url))
        .ToArray();
    }
    
}


### Create the Kernel

In [4]:
// Note: Added this because I am having problems with SSL certificate validation
var handler = new HttpClientHandler();
handler.CheckCertificateRevocationList = false;
var httpClient = new HttpClient(handler);

In [5]:
#pragma warning disable SKEXP0010
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.AddAzureOpenAIChatCompletion(
    deploymentName: deploymentName,
    endpoint: endpoint,
    apiKey: apiKey,
    httpClient: httpClient)
.AddAzureOpenAITextEmbeddingGeneration(adaDeploymentName, endpoint, apiKey, httpClient: httpClient)
.Plugins.AddFromType<YouTubePlugin>("YouTube");

Kernel kernel= kernelBuilder.Build();

// Enable planning
AzureOpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

### Use the plugin in a chat

In [6]:
Dictionary<string, ChatHistory> chats = new()
{
    { "user", new() },
};

// Create a history store the conversation
var history = chats["user"];
history.AddUserMessage("List the lamps");
string? userInput;
while(true)
{
    userInput = await InteractiveKernel.GetInputAsync("User: ");
    if (string.IsNullOrEmpty(userInput))
    {
        break;
    }
    history.AddUserMessage(userInput);
    var result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel,
        cancellationToken: CancellationToken.None);
    history.AddMessage(result.Role, result.Content ?? string.Empty);
    Console.WriteLine($"Assistant: {result.Content}");    
}


Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 92
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 45
   at Submission#6.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)